In [2]:
import numpy as np
from numba import jit, njit, vectorize, cfunc
from timeit import default_timer as timer

In [2]:
1

1

In [3]:
name_=['example4','example5','bin','rome']
name_n=3
Re = np.genfromtxt(name_[name_n]+'\\Re.csv', delimiter=',')
epsD = np.genfromtxt(name_[name_n]+'\\epsD.csv', delimiter=',')
Re

array([120027.16267111,   3729.48906014, 124198.45916958, ...,
         1225.35103377,    985.52416782,    354.43513416])

In [4]:
Np=len(Re)

Re_lam=np.where(Re<=2000,1,0)
Re_turb=np.where(Re>=4000,1,0)
Re_trans=np.ones(Np)-Re_lam-Re_turb

print('fraction of laminar pipes',np.sum(Re_lam)/Np)
print('fraction of transient pipes',np.sum(Re_trans)/Np)
print('fraction of turbulent pipes',np.sum(Re_turb)/Np)

fraction of laminar pipes 0.2709246051961284
fraction of transient pipes 0.11107361181864493
fraction of turbulent pipes 0.6180017829852267


In [44]:
#####################
# friction functions#
#####################

@njit(parallel=True)
def f_swamee(epsD,Re):
    y1=5.74/np.power(Re,0.9)
    y2=epsD/3.7+y1
    c1=-2/np.log(10)
    y3=c1*np.log(y2)
    f=1/(y3*y3)

    y3=c1*np.log(y2)
    fp=(2*0.9*c1)*y1*f/(y2*y3*Re)
    
    return [f,fp]

@njit(parallel=True)
def f_dunlop(epsD,Re):
    aa=-1.5634601348517065795
    #aa=-2*0.9*2/np.log(10)
    ab=0.00328895476345399058690
    #ab=5.74/np.power(4000,0.9)
    c2=-2/np.log(10)
    #c2=-0.8685889638065035
    
    ac=aa*ab
    
    y2=epsD/3.7+ab
    y3=c2*np.log(y2)
    
    fa=1/(y3*y3)
    fb=(2+ac/(y2*y3))*fa
    
    x1=7*fa-fb
    x2=0.128-17*fa+2.5*fb
    x3=-0.128+13*fa-(fb+fb)
    x4=0.032-3*fa+0.5*fb
    
    R=Re/2000
      
    f=x1+R*(x2+R*(x3+R*x4))
    fp=x2+R*(2*x3+3*R*x4)
    fp=fp/2000
    
    return [f,fp]

@njit(parallel=True)
def f_lam(epsD,Re):
    return [64/Re ,-64/(Re*Re)]

@njit(parallel=True)
def f_tot(epsD_vec,Re_vec):
    N=len(Re_vec)
    f_vec=np.zeros(N)
    fp_vec=np.zeros(N)
    for i in range(N):
        Re=Re_vec[i]
        epsD=epsD_vec[i]
        if Re<=2000:
            #############
            #laminar flow
            f=64/Re 
            fp=-64/(Re*Re)
        elif Re>=4000:
            #################
            ###turbulent flow
            #swamee jain#####
            y1=5.74/np.power(Re,0.9)
            y2=epsD/3.7+y1
            c1=-2/np.log(10)
            y3=c1*np.log(y2)
            f=1/(y3*y3)

            y3=c1*np.log(y2)
            fp=(2*0.9*c1)*y1*f/(y2*y3*Re)
        else:
            ######################
            # transition flow####
            #dunlop interpolation
            ######################
            aa=-1.5634601348517065795
            #aa=-2*0.9*2/np.log(10)
            ab=0.00328895476345399058690
            #ab=5.74/np.power(4000,0.9)
            c2=-2/np.log(10)
            #c2=-0.8685889638065035

            ac=aa*ab

            y2=epsD/3.7+ab
            y3=c2*np.log(y2)

            fa=1/(y3*y3)
            fb=(2+ac/(y2*y3))*fa

            x1=7*fa-fb
            x2=0.128-17*fa+2.5*fb
            x3=-0.128+13*fa-(fb+fb)
            x4=0.032-3*fa+0.5*fb

            R=Re/2000

            f=x1+R*(x2+R*(x3+R*x4))
            fp=x2+R*(2*x3+3*R*x4)
            fp=fp/2000
        f_vec[i]=f
        fp_vec[i]=fp
    return [f_vec,fp_vec]


In [45]:
start = timer()
oo=np.ones(len(Re))

f=np.zeros(Np)
fp=np.zeros(Np)

Re_turb=np.zeros(Np)
Re_trans=np.zeros(Np)
Re_lam=np.zeros(Np)

a=np.zeros((Np,2))
a1=np.zeros((Np,2))
a2=np.zeros((Np,2))
a3=np.zeros((Np,2))

print('initialize variables, time=',timer() - start,'s')

initialize variables, time= 0.09249620000002778 s


In [46]:
start = timer()

Re_turb=np.maximum(Re,4000)
Re_trans=np.minimum(np.maximum(Re,2000),4000)
Re_lam=np.minimum(Re,2000)



a_turb=1*(Re>=4000)
a_trans=(Re<4000)*(Re>2000)*1
a_lam=1*(Re<=2000)




a1=f_swamee(epsD,Re)
a2=f_dunlop(epsD,Re)
a3=f_lam(epsD,Re)

a=a_turb*a1+a_trans*a2+a_lam*a3

a=a_turb*f_swamee(epsD,Re)+a_trans*f_dunlop(epsD,Re)+a_lam*f_lam(epsD,Re)


f=a[0]
fp=a[1]
print('Calculate f & fp, time=',timer() - start,'s')

%timeit a=np.where(Re>=4000,1,0)*f_swamee(epsD,Re)+np.where(abs(Re-3000)<1000,1,0)*f_dunlop(epsD,Re)+np.where(Re<=2000,1,0)*f_lam(epsD,Re)

Calculate f & fp, time= 4.941595199999938 s
1.05 s ± 60.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [47]:

a=f_tot(epsD,Re)

start = timer()
a=f_tot(epsD,Re)
f=a[0]
fp=a[1]
print('Calculate f & fp, time=',timer() - start,'s')
%timeit f_tot(epsD,Re)

Calculate f & fp, time= 0.7687401000000591 s
742 ms ± 5.02 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [48]:
Np=10000000
Re=100*np.power(1e8/100,np.random.rand(Np))
epsD=1e-6*np.power(0.05/1e-6,np.random.rand(Np))

In [49]:
%timeit f_tot(epsD,Re)
%timeit np.where(Re>=4000,1,0)*f_swamee(epsD,Re)+np.where(abs(Re-3000)<1000,1,0)*f_dunlop(epsD,Re)+np.where(Re<=2000,1,0)*f_lam(epsD,Re)

752 ms ± 8.44 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
1.06 s ± 39.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [55]:
start = timer()
a=f_tot(epsD,Re)
print(timer() - start)

start = timer()
a1=f_swamee(epsD,Re)
a2=f_dunlop(epsD,Re)
a3=f_lam(epsD,Re)
a10=np.where(Re>=4000,1,0)
a20=np.where(abs(Re-3000)<1000,1,0)
a30=np.where(Re<=2000,1,0)
b=a1*a10+a2*a20+a3*a30
print(timer() - start)



0.7280911000000287
1.0134945999999445


In [54]:
a-b


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

183 ms ± 2.68 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [5]:
import numpy as np
from numba import jit, njit, vectorize, cfunc

In [6]:
@njit(parallel=True)
def f(x_vec):
    N=len(x_vec)
    y_vec=np.zeros(N)
    for i in range(N):
        x=x_vec[i]
        if x<=2000:
            y=64/x
        elif x>=4000:
            y=np.log(x)
        else:
            y=np.log(1.2*x)
        y_vec[i]=y
    return y_vec

In [7]:
Np=10000000
x_vec=100*np.power(1e8/100,np.random.rand(Np))
%timeit f(x_vec)  #0.298 seconds

183 ms ± 3.59 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
